In [ ]:
# 安装watermark
!pip install watermark

In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2022-01-12T23:56:16.671918+08:00

Python implementation: CPython
Python version       : 3.8.10
IPython version      : 7.19.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-91-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 1
Architecture: 64bit



In [3]:
import numpy as np

In [5]:
%watermark --iversions

numpy   : 1.19.5
json    : 2.0.9
autopep8: 1.5.4



## 常量

NumPy 中自带一部分常用的常量，方便直接使用。

### 特殊值

In [5]:
# 自然对数
np.e

2.718281828459045

In [12]:
# PI
np.pi

3.141592653589793

In [96]:
# 0
np.PZERO

0.0

In [97]:
# -0
np.NZERO

-0.0

In [102]:
# None
np.newaxis

### 空值

In [13]:
# 空值
np.nan

nan

In [39]:
type(np.nan)

float

注意，`np.nan` 是一个值，两个 `np.nan` 不相等，虽然它们同属于一个类型。

In [17]:
np.nan is np.nan

True

In [18]:
np.nan == np.nan

False

可以使用 `np.isnan` 方法进行判断。

In [34]:
np.isnan(1), np.isnan(2.0), np.isnan(np.nan), np.isnan(np.log(-10.))

<ipython-input-34-1060db748605>:1: RuntimeWarning: invalid value encountered in log
  np.isnan(1), np.isnan(2.0), np.isnan(np.nan), np.isnan(np.log(-10.))


(False, False, True, True)

In [71]:
# 以下等价
np.nan is np.NAN is np.NaN

True

### 无穷

In [83]:
# 正无穷
np.inf

inf

In [87]:
# 负无穷
np.NINF == -np.inf

True

In [38]:
type(np.inf)

float

In [93]:
np.log(0)

<ipython-input-93-f6e7c0610b57>:1: RuntimeWarning: divide by zero encountered in log
  np.log(0)


-inf

In [42]:
-np.inf < -100

True

In [43]:
np.inf < 10

False

可以使用 `np.isxx` 进行判断。

In [76]:
# 是否正或负去穷
np.isinf(-np.inf)

True

In [77]:
# 哪些元素正无穷
np.isposinf(-np.inf)

False

In [78]:
# 哪些元素负无穷
np.isneginf(np.inf)

False

In [79]:
# 哪些元素有限的（不是非数字、正无穷或负无穷）
np.isfinite(3)

True

In [82]:
np.isfinite(np.inf)

False

In [98]:
# 以下几个方法等价
np.inf == np.Inf == np.Infinity == np.infty == np.PINF 

True

## 标量

![](https://www.numpy.org.cn/static/images/dtype-hierarchy.png)

## 数据类型

数据类型描述了如何解释与数组项对应的固定大小的内存块中的字节。 

- 数据类型
- 数据大小
- 数据顺序
- 如果是「结构化数据类型」则是其他数据类型的集合
- 如果数据类型是子数组，它的形状和数据类型

之前咱们创建 array 的时候都没有关心过数据类型，彼时，numpy 会自动匹配当前输入最合适的数据类型，并将其 cast 到所有元素。这里咱们看下 numpy 的数据类型。

numpy 支持丰富的数据类型，总的来说有以下几种：

- int
- float
- str
- complex


Tips：建议在创建 array 时指定数据类型，且使用统一的数据类型计算。

In [21]:
%timeit np.arange(100, dtype=np.float32).reshape(10, 10) * np.arange(100, dtype=np.int32).reshape(10, 10)

27 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [20]:
%timeit np.arange(100, dtype=np.int32).reshape(10, 10) * np.arange(100, dtype=np.int32).reshape(10, 10)

19.9 µs ± 2.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
# 默认是 int64，也就是 64 位
arr = np.array([1], dtype=np.int64)
arr.dtype

dtype('int64')

In [7]:
# 一共是 8 个字节（Byte），64 位（bit）
# 注意，这里是 16 进制，每个数字是 4 位，两个 16 进制数字就是 8 位
bytes(arr)

b'\x01\x00\x00\x00\x00\x00\x00\x00'

## 数组对象

### ndarray

NumPy 提供了一个 N 维数组类型，即 `ndarray`，描述了**相同类型**「元素」集合。它是偏底层的 array 接口。

所有的 `ndarray` 元素都是同质的，每个元素占用大小相同的内存块，具体大小由「数据类型」决定。`ndarray` 可以共享相同数据。

对象签名如下：

```python
numpy.ndarray(shape, dtype=float, buffer=None, offset=0, strides=None, order=None)
```

- shape：整数元组，表示形状
- dtype：数据类型对象
- buffer：使用 buffer 中的数据填充 `ndarray`
- offset：buffer 中的偏移量
- stride：内存中数据跨度
- order：行为主（C-Style）或列为主（Fortran-Style）


当 buffer 为空时，shape, dtype 和 order 三个参数会被使用；  
当 buffer 不为空时，所有参数都会被使用。

In [358]:
# buffer 为空，结果随机
arr = np.ndarray(shape=(2, 3), dtype=np.int, order="C")
arr

array([[38203952,        0,        0],
       [       0,        0,        0]])

In [394]:
# buffer 不为空
buf = np.array([1, 2, 3, 4], dtype=np.int)
arr = np.ndarray(shape=(2, 2), 
                 dtype=np.int, 
                 offset=0,
                 buffer=buf,
                 strides=(np.int_().itemsize, np.int_().itemsize),
                 )
arr

array([[1, 2],
       [2, 3]])

In [398]:
# buf 的 shape 并无关系
buf = np.array([[[[1, 2]], [[3, 4]], [[5, 6]], [[7, 8]]]], dtype=np.int, order="C")
arr = np.ndarray(shape=(2, 2), 
                 dtype=np.int, 
                 offset=0,
                 buffer=buf,
                 strides=(np.int_().itemsize, np.int_().itemsize),
                 )
arr

array([[1, 2],
       [2, 3]])

In [399]:
# buf 的 order 有关，原因我们后面解释
buf = np.array([[[[1, 2]], [[3, 4]], [[5, 6]], [[7, 8]]]], dtype=np.int, order="F")
arr = np.ndarray(shape=(2, 2), 
                 dtype=np.int, 
                 offset=0,
                 buffer=buf,
                 strides=(np.int_().itemsize, np.int_().itemsize),
                 )
arr

array([[1, 3],
       [3, 5]])

由于我们平时很少用到这个接口，您可能会对其中的一些参数有些困惑。接下来我们稍微解释一下。buffer 为空时没有太多要强调的，重点说一下 buffer 不为空时。

`shape` 和 `dtype` 也比较清晰，`buffer` 刚刚也说明了，使用一位数组即可。主要是剩下的三个参数：`offset`, `strides` 和 `order`。

`order` 是指采用哪种风格进行存储。计算中，行主序（C Style）和列主序（F Style）是将多维数组存储在线性存储器（例如 RAM）中的方法。在行主序中，一行的连续元素彼此相邻，而在列主序中，一列连续元素彼此相邻。具体可参考：[Row- and column-major order - Wikipedia](https://en.wikipedia.org/wiki/Row-_and_column-major_order)。

**需要注意的是**：不同的存储方式会导致计算效率的不同，可以针对具体的场景（处理行多还是列多）选择不同的 Style。

In [289]:
carr = np.arange(1000000).reshape(1000, 1000)
carr.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [290]:
farr = np.asfortranarray(carr)
farr.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [303]:
# 加第一行所有列
# C style 应该比 F style 快一些
%timeit np.sum(carr[0,:])

30.2 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [304]:
%timeit np.sum(farr[0,:])

38.8 µs ± 3.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [306]:
# 加第一列所有行
# F style 应该比 C style 快一些
%timeit np.sum(farr[:, 0])

30.4 µs ± 2.08 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [307]:
%timeit np.sum(carr[:, 0])

34.8 µs ± 1.26 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


`offset` 和 `strides` 是配合使用的，前者是偏移位置，后者是步幅，它必须与 shape 等长。也就是根据给定的 buffer，生成目标 shape 的 array。至于这么做的原因，主要是和内部存储有关，事实上，`ndarray` 就是通过这两个参数来控制 shape，不同的 shape 其实存储是一样的。

In [526]:
buf = np.arange(1, 9)
buf

array([1, 2, 3, 4, 5, 6, 7, 8])

In [532]:
# 因为咱们是 int64，所以 1 个数字是 64 位，即 8 个 Bytes
buf.strides

(8,)

In [528]:
# 这个例子 偏移了「1」个数字，步幅也正好是「1」
# 结果是 从 2 开始
# strides 两个数字分别控制 行和列 的步幅：从左往右看，每次增加 1 个数字，从上往下看，每次增加 1 个数字
arr1 = np.ndarray(
    shape=(2, 3), 
    dtype=np.int8, 
    offset=8,
    buffer=buf,
    strides=(8, 8),
    order="C")
arr1

array([[2, 3, 4],
       [3, 4, 5]], dtype=int8)

In [530]:
# 再来个例子
# 没有偏移，ok，从 1 开始
# 从左到右是列，每次加 1 个数字，从上到下是行，每次增加 2 个数字
arr2 = np.ndarray(
    shape=(3, 2), 
    dtype=np.int8, 
    offset=0,
    buffer=buf,
    strides=(16, 8),
    order="C")
arr2

array([[1, 2],
       [3, 4],
       [5, 6]], dtype=int8)

ok，接下来，我们看一下不同的 shape 的存储情况。

In [537]:
buf = np.arange(1, 9, dtype=np.int8)
buf

array([1, 2, 3, 4, 5, 6, 7, 8], dtype=int8)

In [538]:
# int8 的，每次正好 8 位，即 1 个 Byte
buf.strides

(1,)

In [539]:
bytes(buf)

b'\x01\x02\x03\x04\x05\x06\x07\x08'

In [540]:
# 改一下 shape
buf.shape = 2, 4

In [542]:
buf.strides

(4, 1)

In [541]:
# 发现规律了吗？
buf

array([[1, 2, 3, 4],
       [5, 6, 7, 8]], dtype=int8)

In [544]:
# 此时再看 内存布局
# 和之前是一样的，也就是说，用 strides 我们就可以给同一个 array 不同的 shape
bytes(buf)

b'\x01\x02\x03\x04\x05\x06\x07\x08'

事实上，无论 shape 怎么变化，内存是完全没变化的，不同的 array 其实就是不同的 strides 方式而已。感兴趣的可以进一步尝试。

另外，使用 buffer 创建的 `ndarray` 都使用了同一块内存。

In [565]:
buf = np.arange(1, 9, dtype=np.int8)

In [566]:
arr1 = np.ndarray(
    shape=(2, 3), 
    dtype=np.int8, 
    offset=0,
    buffer=buf,
    strides=(1, 1),
    order="C")
arr1

array([[1, 2, 3],
       [2, 3, 4]], dtype=int8)

In [567]:
arr2 = np.ndarray(
    shape=(3, 2), 
    dtype=np.int8, 
    offset=0,
    buffer=buf,
    strides=(1, 1),
    order="C")
arr2

array([[1, 2],
       [2, 3],
       [3, 4]], dtype=int8)

In [574]:
bytes(arr1), bytes(arr2)

(b'\x01\x02\x03\x02\x03\x04', b'\x01\x02\x02\x03\x03\x04')

In [575]:
np.may_share_memory(arr1, arr2), np.may_share_memory(buf, arr1)

(True, True)

其实，无论 arr1 还是 arr2 都是 buf 的一个 view（引用），与此相对的是 copy。一般来说，切片（slicing）会创建 view，索引（indexing）会创建 copy。我们在后面的内容会进一步解释。

也就是说，使用 buffer 创建 `ndarray` 其实可以理解成一种「切片」。实际上，如果您查看 `np.array` 的接口，就会发现其中有个 `copy` 参数，它默认是 `True`。

In [579]:
buf, arr1, arr2

(array([1, 2, 3, 4, 5, 6, 7, 8], dtype=int8),
 array([[1, 2, 3],
        [2, 3, 4]], dtype=int8),
 array([[1, 2],
        [2, 3],
        [3, 4]], dtype=int8))

In [580]:
buf[0] = 9

In [581]:
buf, arr1, arr2

(array([9, 2, 3, 4, 5, 6, 7, 8], dtype=int8),
 array([[9, 2, 3],
        [2, 3, 4]], dtype=int8),
 array([[9, 2],
        [2, 3],
        [3, 4]], dtype=int8))

`strides`　不同时，处理的效率也有差异。当步长增加时，找到对应位置的值会变慢。其原因是，CPU 在处理任务时会将数据从内存读取到缓存，步长小时，需要的传输更少。比如要取 10 个数，连在一起的（步长=1个数字）可以一次取到，但步长大时却要取多次。

>注：CPU 缓存是用于减少处理器访问内存所需平均时间的部件。在金字塔式存储体系中它位于自顶向下的第二层，仅次于 CPU 寄存器。其容量远小于内存，但速度却可以接近处理器的频率。一般会有多级缓存。——维基百科

In [628]:
arr1 = np.ones((1000, 100), dtype=np.int8)
arr2 = np.ones((10000, 100), dtype=np.int8)[::10]
arr1.shape, arr2.shape

((1000, 100), (1000, 100))

In [639]:
arr1.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [638]:
# OWNDATA=False，意思是这个 array 的数据是从其他地方「借」来的
# 从哪个地方呢？当然就是 `np.ones((10000, 100), dtype=np.int8)` 这里了
arr2.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [631]:
np.any(arr1 == arr2)

True

In [632]:
arr1.strides, arr2.strides

((100, 1), (1000, 1))

In [633]:
%timeit arr1.sum()

673 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [634]:
%timeit arr2.sum()

785 µs ± 8.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


最后说明下，我们上面的例子都是用 `int` 类型来说明，其他数据类型类似。

另外，源码中的大致逻辑是：`arrayobject.c` 中的 `array_new` 方法调用了 ctors.c 中的 `PyArray_NewFromDescr_int` 来实现创建一个 `ndarray`。

### array

首先要明确，`array` 只是快速创建 `ndarray` 的接口函数，源代码是 `core/src/multiarray/methods.c` 中的 `array_getarray`。这玩意儿其实就调用了上面提到的 `PyArray_NewFromDescr_int`。

## 小结


- `ndarray` 是基本的数组对象，它底层有两种存储方式（C Style 和 F Style），可以根据实际处理逻辑选择合适的方式。另外注意，切片后的引用，步长长的比步长短的处理起来要慢。